In [1]:
importall Base

In [2]:
include("frames.jl");

In [3]:
abstract CacheElement{T}

immutable ImmutableCacheElement{T} <: CacheElement{T}
    data::T
end

get{T}(element::ImmutableCacheElement{T}) = element.data
function setdirty!{T}(element::ImmutableCacheElement{T})
end

type MutableCacheElement{T} <: CacheElement{T}
    data::T
    updateFunction::Function
    dirty::Bool
    
    MutableCacheElement(data::T, updateFunction::Function) = new(data, updateFunction, true)
end
MutableCacheElement{T}(data::T, updateFunction::Function) = MutableCacheElement{T}(data, updateFunction)
MutableCacheElement(updateFunction::Function) = MutableCacheElement(updateFunction(), updateFunction)

function get{T}(element::MutableCacheElement{T})
    if element.dirty
        element.data = element.updateFunction()
        element.dirty = false
    end
    return element.data
end
function setdirty!{T}(element::MutableCacheElement{T})
    element.dirty = true
end

setdirty! (generic function with 2 methods)

In [4]:
type FrameCache{T}
    root::CartesianFrame3D
    transformsToParent::Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}
    transformsToWorld::Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}
    dependencies::Dict{CacheElement{Transform3D{T}}, Vector{CacheElement{Transform3D{T}}}}
    
    function FrameCache(root::CartesianFrame3D)
        transformsToParent = Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}()
        transformsToWorld = Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}(root => ImmutableCacheElement(Transform3D{T}(root, root)))
        dependencies = Dict{CacheElement{Transform3D{T}}, Vector{CacheElement{Transform3D{T}}}}()
        new(root, transformsToParent, transformsToWorld, dependencies)
    end
end

In [55]:
function add_frame!{T}(cache::FrameCache{T}, t::Transform3D{T})
    # for fixed frames
    to_parent = ImmutableCacheElement(t)
    cache.dependencies[to_parent] = []
    parent_to_world = cache.transformsToWorld[t.to]
    cache.transformsToParent[t.from] = to_parent

    updateTransformToWorld = () -> begin println("updating world 1"); get(parent_to_world) * get(to_parent) end
    to_world = MutableCacheElement(updateTransformToWorld)
    cache.dependencies[to_world] = [parent_to_world]
    cache.transformsToWorld[t.from] = to_world
    return cache
end

function add_frame!{T}(cache::FrameCache{T}, updateTransformToParent::Function, dependencies...)
    # for non-fixed frames
    to_parent = MutableCacheElement(() -> begin println("updating parent"); updateTransformToParent() end)
    cache.dependencies[to_parent] = [cache.transformsToWorld[x] for x in dependencies]
    t = to_parent.data
    parent_to_world = cache.transformsToWorld[t.to]
    cache.transformsToParent[t.from] = to_parent
    
    updateTransformToWorld = () -> begin println("updating world 2"); get(parent_to_world) * get(to_parent) end
    to_world = MutableCacheElement(updateTransformToWorld)
    cache.dependencies[to_world] = [to_parent, parent_to_world]
    cache.transformsToWorld[t.from] = to_world
    return cache
end

function setdirty!{T}(cache::FrameCache{T}, element::CacheElement{Transform3D{T}})
    setdirty!(element)
    for dependent in cache.dependencies
        setdirty!(cache, element)
    end
end

function setdirty!{T}(cache::FrameCache{T})
    for element in values(cache.transformsToParent)
        setdirty!(element)
    end
    for element in values(cache.transformsToWorld)
        setdirty!(element)
    end
end

function transform_to_parent{T}(cache::FrameCache{T}, frame::CartesianFrame3D)
    frame == cache.root && error("frame has no parent")
    return get(cache.transformsToParent[frame])
end

transform_to_world{T}(cache::FrameCache{T}, frame::CartesianFrame3D) = get(cache.transformsToWorld[frame])

function relative_transform{T}(cache::FrameCache{T}, from::CartesianFrame3D, to::CartesianFrame3D)
    return inv(transform_to_world(cache, to)) * transform_to_world(cache, from)
end

relative_transform (generic function with 1 method)

In [56]:
root = CartesianFrame3D("root")
cache = FrameCache{Float64}(root)
f1 = CartesianFrame3D("f1")
t1 = Transform3D(f1, root, nquatrand(), rand(Vec{3, Float64}))
add_frame!(cache, t1)
f2 = CartesianFrame3D("f2")
add_frame!(cache, () -> Transform3D(f2, f1, nquatrand(), rand(Vec{3, Float64})))

updating world 1
updating parent
updating world 2
updating world 1
updating parent


FrameCache{Float64}(CartesianFrame3D("root"),Dict{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(CartesianFrame3D("f2")=>MutableCacheElement{Transform3D{Float64}}(Transform3D{Float64}(CartesianFrame3D("f2"),CartesianFrame3D("f1"),0.3263027384390807 + 0.7257185220062254im - 0.575483662713472jm + 0.18888542467364555km,FixedSizeArrays.Vec{3,Float64}((0.914163544345731,0.6743368132745888,0.5999454344908846))),(anonymous function),false),CartesianFrame3D("f1")=>ImmutableCacheElement{Transform3D{Float64}}(Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),-0.5770582320694566 - 0.5830858599631293im - 0.1762783673750573jm - 0.5440042407072903km,FixedSizeArrays.Vec{3,Float64}((0.05219971236289034,0.7910946792126832,0.7926672482707788))))),Dict{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(CartesianFrame3D("f2")=>MutableCacheElement{Transform3D{Float64}}(Transform3D{Float64}(CartesianFrame3D("f2"),CartesianFrame3D("root"),0.23616967864820457 - 0.9554063273963542im 

In [74]:
transform_to_parent(cache, f1)

Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),-0.5770582320694566 - 0.5830858599631293im - 0.1762783673750573jm - 0.5440042407072903km,FixedSizeArrays.Vec{3,Float64}((0.05219971236289034,0.7910946792126832,0.7926672482707788)))

In [75]:
transform_to_world(cache, f1)

updating world 1


Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),-0.5770582320694566 - 0.5830858599631293im - 0.1762783673750573jm - 0.5440042407072903km,FixedSizeArrays.Vec{3,Float64}((0.05219971236289034,0.7910946792126832,0.7926672482707788)))

In [76]:
relative_transform(cache, root, f1)

Transform3D{Float64}(CartesianFrame3D("root"),CartesianFrame3D("f1"),-0.5770582320694566 + 0.5830858599631293im + 0.1762783673750573jm + 0.5440042407072903km,FixedSizeArrays.Vec{3,Float64}((-1.018974995809246,-0.44834301735470294,0.1324970363267902)))

In [77]:
relative_transform(cache, root, f2)

updating world 2
updating parent


Transform3D{Float64}(CartesianFrame3D("root"),CartesianFrame3D("f2"),-0.839318965096296 - 0.09498241208655235im - 0.3978017823869778jm + 0.3581560527949333km,FixedSizeArrays.Vec{3,Float64}((-2.1115276986230564,0.35820727869069435,-0.05780412133398116)))

In [73]:
setdirty!(cache)

In [ ]:
# J = rand(Mat{3, 3, Float64}); J = J * J'
# c = rand(Vec{3, Float64})
# m = rand()
# inertia = SpatialInertia(f3, J, c, m)
# m = Mechanism()
# world = add_body!(m, RigidBody("world"))
# body = add_body!(m, RigidBody("body", inertia))
# add_joint!(m, QuaternionFloatingJoint("joint"), world, body)

In [ ]:
# transform(inertia, t3)

In [5]:
[()]

1-element Array{Tuple{},1}:
 ()

In [6]:
((1, 2), 3)

((1,2),3)

In [8]:
tuple((1,2)...,(3)...)

(1,2,3)

In [12]:
ImmutableCacheElement(32.)

ImmutableCacheElement{Float64}(32.0)

In [16]:
dependencies = (ImmutableCacheElement(32.), MutableCacheElement(() -> 3), MutableCacheElement(() -> 3))

(ImmutableCacheElement{Float64}(32.0),MutableCacheElement{Int64}(3,(anonymous function),true),MutableCacheElement{Int64}(3,(anonymous function),true))

In [12]:
[dependencies...]

2-element Array{CartesianFrame3D,1}:
 CartesianFrame3D("root")
 CartesianFrame3D("f1")  

In [17]:
typeof(dependencies)

Tuple{ImmutableCacheElement{Float64},MutableCacheElement{Int64},MutableCacheElement{Int64}}

In [19]:
@which Tuple{}()

call{T}(::Type{T}, args...) at essentials.jl:57